# 1) Import the data

In [4]:
import pandas as pd

jan_df = pd.read_parquet('../data/fhv_tripdata_2021-01.parquet' ,engine='pyarrow')
feb_df = pd.read_parquet('../data/fhv_tripdata_2021-02.parquet' ,engine='pyarrow')

# Q1 - Read the data for January. How many records are there?
- [ ]1054112
- [X]1154112 
- [ ]1254112
- [ ]1354112

In [6]:
print(f"January has: {len(jan_df)} records.")

January has: 1154112 records


In [10]:
print(f"Features: {jan_df.columns}")

Features: Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'],
      dtype='object')


In [11]:
jan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1154112 entries, 0 to 1154111
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1154112 non-null  object        
 1   pickup_datetime         1154112 non-null  datetime64[ns]
 2   dropOff_datetime        1154112 non-null  datetime64[ns]
 3   PUlocationID            195845 non-null   float64       
 4   DOlocationID            991892 non-null   float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1153227 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 61.6+ MB


In order to calculate the duration we should only subtract `dropOff_datetime` from `pickup_datetime`.

In [18]:
jan_df['duration'] = jan_df['dropOff_datetime'] - jan_df['pickup_datetime']
jan_df['duration'] = jan_df['duration'].dt.total_seconds() / 60.

# 1.1) Analyze duration

# Q2 - What's the average trip duration in January?

- [ ] 15.16
- [X] 19.16
- [ ] 24.16
- [ ] 29.16


In [23]:
jan_df['duration'].describe(percentiles=[0.95, 0.98, 0.99])

count    1.154112e+06
mean     1.916722e+01
std      3.986922e+02
min      1.666667e-02
50%      1.340000e+01
95%      4.725000e+01
98%      6.613333e+01
99%      9.030000e+01
max      4.233710e+05
Name: duration, dtype: float64

we should focus on durations that are at least one minute and less than 60 minutes.

In [24]:
mask = (jan_df['duration'] >=1) & (jan_df['duration'] <= 60)
jan_df_hour = jan_df[mask]

# Q3 - What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

- [ ] 53%
- [ ] 63%
- [ ] 73%
- [X] 83%

In [29]:
categorical = ['PUlocationID', 'DOlocationID']
numerical = ['duration']
features = categorical + numerical
jan_df_train = jan_df_hour[features]
jan_df_train.isna().sum() / len(jan_df_train)

PUlocationID    0.835273
DOlocationID    0.133270
duration        0.000000
dtype: float64

Fill NA with `-1`

In [30]:
jan_df_train[jan_df_train.isna()]['PUlocationID'] = -1
jan_df_train[jan_df_train.isna()]['DOlocationID'] = -1
jan_df_train.isna().sum() / len(jan_df_train)

PUlocationID    0.835273
DOlocationID    0.133270
duration        0.000000
dtype: float64

In [33]:
jan_df_train = jan_df_train.fillna(-1)
jan_df_train.isna().sum() / len(jan_df_train)

PUlocationID    0.0
DOlocationID    0.0
duration        0.0
dtype: float64